In [ ]:
# import sys
# sys.path.insert(0, '/Users/niloufar/Desktop/mlops/mlops')


In [ ]:
pip install tfx

In [3]:
import tensorflow as tf
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import pandas as pd

import os
import pprint

In [ ]:
# import inspect
# inspect.getfile(tensorflow_model_analysis)

In [ ]:
# pip install ml-metadata
# pip install apache-beam

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pp = pprint.PrettyPrinter()

In [5]:
_pipeline_root = './pipeline/'
_data_root = '/content/drive/MyDrive/mlops/course2/week2/data/'
_data_filepath = os.path.join(_data_root, 'adult.csv')

In [ ]:
# df = pd.read_csv(_data_filepath, skipinitialspace=True)

In [ ]:
# headerList = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label']

In [6]:
df = pd.read_csv(_data_root +"adult.csv", skipinitialspace=True)

In [7]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
# df.loc[-0.5] = [39, 
#                 'State-gov', 
#                  77516, 
#                 'Bachelors', 
#                  13, 
#                 'Never-married', 
#                 'Adm-clerical', 
#                 'Not-in-family', 
#                 'White', 
#                 'Male', 
#                  2174, 
#                  0, 
#                  40, 
#                 'United-States', 
#                  '<=50K']
# df = df.sort_index().reset_index(drop=True)

In [ ]:
# df.to_csv( _data_root +"adult.csv", header=headerList, index=False)
# df.head()

In [8]:
!head {_data_filepath}

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husban

In [9]:
context = InteractiveContext(pipeline_root=_pipeline_root)

In [10]:
example_gen = CsvExampleGen(input_base=_data_root)

In [11]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [12]:
artifact = example_gen.outputs['examples'].get()[0]
artifact.uri, artifact.split_names


('./pipeline/CsvExampleGen/examples/1', '["train", "eval"]')

In [13]:
train_uri = os.path.join(artifact.uri, 'Split-train')
train_uri

'./pipeline/CsvExampleGen/examples/1/Split-train'

In [14]:
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


In [15]:
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')

In [16]:
def get_records(dataset, num_records):

  records = []
  for tfrecord in dataset.take(num_records):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    example_dict = (MessageToDict(example))
    records.append(example_dict)

  return records

In [17]:
sample_records = get_records(dataset, 1)
pp.pprint(sample_records)

[{'features': {'feature': {'age': {'int64List': {'value': ['39']}},
                           'capital-gain': {'int64List': {'value': ['2174']}},
                           'capital-loss': {'int64List': {'value': ['0']}},
                           'education': {'bytesList': {'value': ['QmFjaGVsb3Jz']}},
                           'education-num': {'int64List': {'value': ['13']}},
                           'fnlwgt': {'int64List': {'value': ['77516']}},
                           'hours-per-week': {'int64List': {'value': ['40']}},
                           'label': {'bytesList': {'value': ['PD01MEs=']}},
                           'marital-status': {'bytesList': {'value': ['TmV2ZXItbWFycmllZA==']}},
                           'native-country': {'bytesList': {'value': ['VW5pdGVkLVN0YXRlcw==']}},
                           'occupation': {'bytesList': {'value': ['QWRtLWNsZXJpY2Fs']}},
                           'race': {'bytesList': {'value': ['V2hpdGU=']}},
                           '

In [18]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [19]:
context.show(statistics_gen.outputs['statistics'])

In [20]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics']
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [ ]:
context.show(schema_gen.outputs['schema'])

In [23]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [24]:
context.show(example_validator.outputs['anomalies'])

In [40]:
_census_constants_module_file = 'census_constants.py'

In [42]:

# %%writefile census_constants.py

UsageError: %%writefile is a cell magic, but the cell body is empty.


In [47]:
open(_census_constants_module_file, 'w')

<_io.TextIOWrapper name='census_constants.py' mode='w' encoding='UTF-8'>

In [ ]:
CATEGORICAL_FEATURE_KEYS = [
    'education', 'marita_status', 'occupation', 'race', 'relationship', 'workclass', 'sex', 'native-country'
]

NUMERIC_FEATURE_KEYS = ['fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

BUCKET_FEATURE_KEYS = ['age']
FEATURE_BUCKET_COUNT = {'age': 4} 

LABEL_KEY = 'label'

def transformed_name(key):
  return key + '-xf'